In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from state import get_state, queue, get_state_per_traffic_light
import os
import traci

In [3]:
from dotenv import load_dotenv
load_dotenv()

sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = "double_traffic/double_traffic.sumo.cfg"


#sumo_bin = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
#sumo_gui_bin = os.getenv("SUMO-GUI")
#sumo_gui_bin = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo-gui.exe"
#simulConfig = r"double_traffic/doube_traffic.sumo.cfg"
#sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"



print(sumo_gui_bin)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [4]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = (64,)  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_one_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])
model_action_one_tf_light.summary()

/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/Users/pr/.pyenv/versions/3.10.6/envs/sumo_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,268 (12.77 KB)

 Trainable params: 3,268 (12.77 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
set_seed(42)  # extra code – ensures reproducibility on the CPU
# receives states and actions of both agent 1 and 2
input_shape = [184]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model__critic_action_one_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [7]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = (64,)  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_second_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [8]:
set_seed(42)  # extra code – ensures reproducibility on the CPU
# receives states and actions of both agent 1 and 2
input_shape = [184]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model__critic_second_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [9]:
# def epsilon_greedy_policy(state_tf_one, state_tf_two,epsilon=0):
#     if np.random.rand() < epsilon:
#         return np.random.randint(n_outputs), np.random.randint(n_outputs)  # random action
#     else:
#         Q_values_tf_one = model_action_one_tf_light.predict(state_tf_one[np.newaxis], verbose=0)[0]
#         Q_values_tf_two = model_action_second_tf_light.predict(state_tf_two[np.newaxis], verbose=0)[0]

#         print(f"state : {state_tf_one, state_tf_two}")
#         print(f"Q_values : {Q_values_tf_one, Q_values_tf_two}")
#         return Q_values_tf_one.argmax(), Q_values_tf_two.argmax()  # optimal action according to the DQN

In [10]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(9)
    ]
    return states, rewards, next_states


In [11]:
@tf.function
def train_agent(agent_idx, batch, agents, gamma=0.95):
    # batch = (states, actions, rewards, next_states)
    states, actions, rewards, next_states = batch

    agent = agents[agent_idx]

    ### Critic update ###
    with tf.GradientTape() as tape:
        # Actions suivantes pour chaque agent avec target actor
        next_actions = [agents[i].target_actor(next_states[:, i, :]) for i in range(len(agents))]
        next_actions = tf.concat(next_actions, axis=-1)

        # Critic target Q
        target_Q = agent.target_critic([next_states, next_actions])
        target_Q = rewards[:, agent_idx] + gamma * target_Q

        # Current Q avec critic principal
        current_actions = tf.concat(actions, axis=-1)
        current_Q = agent.critic([states, current_actions])

        critic_loss = tf.reduce_mean(tf.square(target_Q - current_Q))

    critic_grad = tape.gradient(critic_loss, agent.critic.trainable_variables)
    agent.critic_optimizer.apply_gradients(zip(critic_grad, agent.critic.trainable_variables))

    ### Actor update ###
    with tf.GradientTape() as tape:
        # Actions proposées par les actors
        current_actions = []
        for i in range(len(agents)):
            if i == agent_idx:
                current_action = agent.actor(states[:, i, :])
            else:
                current_action = actions[i]  # on stoppe le gradient sur les autres
            current_actions.append(current_action)

        current_actions = tf.concat(current_actions, axis=-1)

        # Maximise Q donc minimise -Q
        actor_loss = -tf.reduce_mean(agent.critic([states, current_actions]))

    actor_grad = tape.gradient(actor_loss, agent.actor.trainable_variables)
    agent.actor_optimizer.apply_gradients(zip(actor_grad, agent.actor.trainable_variables))

    return critic_loss, actor_loss


In [12]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
replay_buffer = deque(maxlen=10000)

epoch = 500






# PERIODE DE STABILITE

# traffic_scale = [0.1] * 100
# AUGMENTATION DE TRAFFIC
# traffic_scale.extend(np.linspace(0.1, 1, 400, dtype=float))
# OR
#traffic_scale = 0.2



for episode in range(epoch):
    #SEED & TRAFFIC SCALING
    seed_value = np.random.randint(1000)
    #seed_value = 41
    if traci.isLoaded():
        traci.close()


    sumoCmd = [sumo_bin, "-c", simulConfig, "--start"]

    traci.start(sumoCmd)





    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    traffic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))

    b = traffic_light_ids[0]

# REMPLACER B PAR LE FEU 2


    state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
    state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
    action_tf_one=-1
    action_tf_two=-1





    # print(state)
    for step in range(80000): ## TO CHANGED
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state_tf_one = np.array(get_state_per_traffic_light(traffic_light_ids[0]))
            next_state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))




            # 4 pour l'action dim car chaque feu à 4 phases
            noise_one = np.random.normal(0, 0.1, size=4)
            noise_two = np.random.normal(0, 0.1, size=4)
            print("shape", state_tf_one.shape)
            print("shape", state_tf_two.shape)
            action_tf_one = model_action_one_tf_light.predict(state_tf_one[np.newaxis]) + noise_one
            action_tf_two = model_action_second_tf_light(state_tf_two[np.newaxis]) + noise_two
            action_tf_one = np.argmax(action_tf_one).reshape(1,1)
            action_tf_two = np.argmax(action_tf_two).reshape(1,1)


            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward_tf_one = np.sum(state_tf_one)- np.sum(next_state_tf_one)
            reward_tf_two = np.sum(state_tf_two)- np.sum(next_state_tf_two)
            reward_tf_one = reward_tf_one.reshape(1,1)
            reward_tf_two = reward_tf_two.reshape(1,1)

            states = np.concat([state_tf_one, state_tf_two])
            actions = np.concat([action_tf_one, action_tf_two])
            rewards = np.concat([reward_tf_one, reward_tf_two])
            next_states = np.concat([next_state_tf_one, next_state_tf_two])


            replay_buffer.append((states, actions, rewards, next_states))
            #########################################################################
            state_tf_one=next_state_tf_one
            state_tf_two = next_state_tf_two


            print("action", action_tf_one)
            traci.trafficlight.setPhase(traffic_light_ids[0],2*int(action_tf_one))
            traci.trafficlight.setPhase(traffic_light_ids[0],2*int(action_tf_two))

            if len(replay_buffer) >= 10:
                 for agent in range(2):
                    train_agent(agent, replay_buffer)

                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
action [[1]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
action [[0]]


/var/folders/2c/jlpbrr154jg9cfp1pzbh8x5h0000gn/T/ipykernel_72344/2183737402.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  traci.trafficlight.setPhase(traffic_light_ids[0],2*int(action_tf_one))
/var/folders/2c/jlpbrr154jg9cfp1pzbh8x5h0000gn/T/ipykernel_72344/2183737402.py:110: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  traci.trafficlight.setPhase(traffic_light_ids[0],2*int(action_tf_two))


shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
action [[1]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
action [[2]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
action [[2]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
action [[2]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
action [[2]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
action [[0]]
shape (64,)
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
action [[2]]
Stepshape (64,)ms ~= 10.00*RT, ~111000.00UPS, TraCI: 0ms, vehicles TOT 228 ACT 111 BUF 3) 
shape (64,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
action [[0]]


TypeError: Binding inputs to tf.function failed due to `missing a required argument: 'agents'`. Received args: (0, deque([(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([[1],
       [3]]), array([[0],
       [0]]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED é SCALE
seed_value = np.random.randint(1000)
#seed_value = 41
traffic_scale = 0.2


#LAUNCH
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--scale', str(traffic_scale), '--gui-settings-file', 'full_state_sumo_gui_settings.xml']

# if traci.isLoaded():
#     traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        #action = np.random.randint(3)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))

        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()
